In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [ ]:
# Dataset Path
dataset_path = 'PNEUMONIACOVID'

In [ ]:
# Fungsi untuk menghitung jumlah gambar dalam setiap folder
def count_images_per_folder(path):
    folder_counts = {}
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            num_images = len([f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))])
            folder_counts[folder_name] = num_images
    return folder_counts

# Hitung jumlah gambar per folder
folder_counts = count_images_per_folder(dataset_path)
print("Jumlah gambar per folder:")
for folder, count in folder_counts.items():
    print(f"{folder}: {count}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(folder_counts.keys(), folder_counts.values(), color=['blue', 'green', 'red'])
plt.title("Distribusi Kelas dalam Dataset")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Gambar")
plt.show()

In [ ]:
# Fungsi untuk menampilkan beberapa contoh gambar per folder
def show_sample_images(path, num_samples=4):
    fig, axes = plt.subplots(len(os.listdir(path)), num_samples, figsize=(15, 10))
    fig.suptitle("Sample Images from Each Class", fontsize=16)
    
    for i, folder_name in enumerate(os.listdir(path)):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
            sample_files = random.sample(image_files, min(num_samples, len(image_files)))
            
            for j, image_file in enumerate(sample_files):
                image_path = os.path.join(folder_path, image_file)
                img = cv2.imread(image_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display
                axes[i, j].imshow(img)
                axes[i, j].axis('off')
                axes[i, j].set_title(f"{folder_name}")
    
    plt.tight_layout()
    plt.show()

# Menampilkan contoh gambar
show_sample_images(dataset_path, num_samples=4)

In [ ]:
img_size = (224, 224)
batch_size = 32

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
# Load Training Data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
# Load Validation Data
validation_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Parameters
epochs = 50

In [ ]:
input_tensor = Input(shape=(224, 224, 3))  # Dimensi harus jelas
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)


# Freeze base model
base_model.trainable = False

In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile Model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('mobilenetv2_best_model.keras', save_best_only=True, monitor='val_loss')

In [ ]:
# Train Model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

In [ ]:
plt.figure(figsize=(18, 6))

# Plot akurasi training dan validation
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Akurasi Training')
plt.plot(history.history['val_accuracy'], label='Akurasi Validation')
plt.title('Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(18, 6))

# Plot loss training dan validation
plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Loss Training')
plt.plot(history.history['val_loss'], label='Loss Validation')
plt.title('Loss Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluation
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Loss pada data validasi: {val_loss:.4f}")
print(f"Akurasi pada data validasi: {val_accuracy:.4f}")

In [ ]:
# Evaluasi model pada data validasi
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Loss pada data validasi: {val_loss:.4f}")
print(f"Akurasi pada data validasi: {val_accuracy:.4f}")

In [ ]:
# Save Model
model_path = 'model.keras'
model.save(model_path)
print(f"Model saved at {model_path}")
